In [1]:
# import packages
import pandas as pd
import numpy as np
import great_expectations as ge
import json

# load data
data = pd.read_excel("hw5_dataset.xlsx")
display(data)

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,Data Scientist,$105K-$167K (Glassdoor est.),Summary\n\nWe‚Äôre looking for a data scientis...,3.6,TRANZACT\n3.6,"Fort Lee, NJ","Fort Lee, NJ",1001 to 5000 employees,1989,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
668,668,Data Scientist,$105K-$167K (Glassdoor est.),Job Description\nBecome a thought leader withi...,-1.0,JKGT,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
669,669,Data Scientist,$105K-$167K (Glassdoor est.),Join a thriving company that is changing the w...,-1.0,AccessHope,"Irwindale, CA",-1,-1,-1,-1,-1,-1,-1,-1
670,670,Data Scientist,$105K-$167K (Glassdoor est.),100 Remote Opportunity As an AINLP Data Scient...,5.0,ChaTeck Incorporated\n5.0,"San Francisco, CA","Santa Clara, CA",1 to 50 employees,-1,Company - Private,Advertising & Marketing,Business Services,$1 to $5 million (USD),-1


In [2]:
## pre-process data for validation

# change all “-1” to missing values before running the validation procedure
data.replace(-1, np.nan, inplace=True)

# add column 'A' for the minimum Salary Estimate and column 'B' for the maximum Salary Estimate (A and B are both in thousands of dollars)
pattern = r'\$(\d+)K\-\$(\d+)K'
data[['A', 'B']] = data['Salary Estimate'].str.extract(pattern)
data['A'] = pd.to_numeric(data['A'])
data['B'] = pd.to_numeric(data['B'])

# add a column 'B - A' to check if A is actually strictly less than B (in thousands of dollars)
data['B - A'] = data['B'] - data['A']

# add a column 'Revenue_2019' that contains the revenue of firms founded in 2019 and contains "Unknown / Non-Applicable"
# if the firm was not founded in 2019 so that it doesn't show up as unexpected value
data['Revenue_2019'] = data['Revenue']
data.loc[data['Founded'] != 2019, 'Revenue_2019'] = "Unknown / Non-Applicable"


In [3]:
# convert data to ge_data
ge_data = ge.from_pandas(data)


# Expectation (1): Salary Estimate are between 50k and 400k
ge_data.expect_column_values_to_be_between('A', min_value=50, max_value = 400)
ge_data.expect_column_values_to_be_between('B', min_value=50, max_value = 400)

# Expectation (2): Salary Estimate are listed as {A}–{B} where A is strictly smaller than B
ge_data.expect_column_values_to_be_between('B - A', min_value=1, max_value = 350)
ge_data.expect_column_values_to_match_regex('Salary Estimate', r'^\$\d+K-\$\d+K.*$')

# Expectation (3): Rating is between 0 and 5
ge_data.expect_column_values_to_be_between('Rating', min_value=0, max_value=5)

# Expectation (4): Location of the company is either remote or in the United States
us_pattern1 = (r'\b(?:AL|AK|AZ|AR|CA|CO|CT|DC|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA'
                r'|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT'
                r'|VT|VA|WA|WV|WI|WY)\b')
us_pattern2 = (r'(?:Alabama|Alaska|Arizona|Arkansas|California|Colorado|Connecticut'
                r'District of Columbia|Delaware|Florida|Georgia|Hawaii|Idaho|Illinois'
                r'Indiana|Iowa|Kansas|Kentucky|Louisiana|Maine|Maryland|Massachusetts'
                r'Michigan|Minnesota|Mississippi|Missouri|Montana|Nebraska|Nevada'
                r'New Hampshire|New Jersey|New Mexico|New York|North Carolina|North Dakota'
                r'Ohio|Oklahoma|Oregon|Pennsylvania|Rhode Island|South Carolina|South Dakota'
                r'Tennessee|Texas|Utah|Vermont|Virginia|Washington|West Virginia|Wisconsin|Wyoming)')

pattern_list = [r'\bremote\b', r'\bRemote\b', r'\bUnited States\b', us_pattern1, us_pattern2]
ge_data.expect_column_values_to_match_regex_list('Location', pattern_list)

# Expectation (5): Size variable contains the pattern {integer} – to – {integer} employees
ge_data.expect_column_values_to_match_regex('Size', r'^\d+ to \d+ employees$')

# Expectation (6): Founded to be later than the year 1800
ge_data.expect_column_values_to_be_between('Founded', min_value=1801, max_value=2024)


# Expectation (7): All firms founded in 2019 should not have Revenue (i.e. Unknown / Non-Applicable)
ge_data.expect_column_values_to_match_regex('Revenue_2019', r'^Unknown / Non-Applicable$')


# print resuls
results = ge_data.validate()

json_results = results.to_json_dict()
print(json.dumps(json_results, indent = 4))


{
    "success": false,
    "results": [
        {
            "success": false,
            "expectation_config": {
                "expectation_type": "expect_column_values_to_be_between",
                "kwargs": {
                    "column": "A",
                    "min_value": 50,
                    "max_value": 400,
                    "result_format": "BASIC"
                },
                "meta": {}
            },
            "result": {
                "element_count": 672,
                "missing_count": 0,
                "missing_percent": 0.0,
                "unexpected_count": 21,
                "unexpected_percent": 3.125,
                "unexpected_percent_total": 3.125,
                "unexpected_percent_nonmissing": 3.125,
                "partial_unexpected_list": [
                    990,
                    31,
                    31,
                    31,
                    31,
                    31,
                    31,
                    3

In [5]:
# output the index of any rows that contain unvalidated data
indices = []
for r in results.results:
    if not r.success:
        col = r.expectation_config.kwargs["column"]
        unexpected = r.result["partial_unexpected_list"]
        indices += data[data[col].isin(unexpected)].index.tolist()        
        
indices = np.unique(indices)
        
display(indices)

array([  6,  31,  41,  45,  51,  60,  73,  80,  89,  90,  97, 108, 109,
       138, 150, 155, 159, 164, 168, 169, 181, 187, 192, 194, 204, 215,
       235, 237, 245, 246, 252, 256, 268, 272, 277, 279, 286, 287, 292,
       295, 299, 300, 321, 334, 336, 347, 354, 365, 370, 372, 383, 395,
       401, 417, 420, 434, 442, 444, 448, 449, 452, 455, 467, 468, 469,
       470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482,
       483, 484, 485, 486, 493, 508, 509, 535, 536, 548, 570, 578, 581,
       589, 591, 594, 599, 607, 611, 614, 617, 631, 644, 645, 651, 659,
       661])